<a href="https://colab.research.google.com/github/dn-bit/using-LSTM/blob/main/Sentiment_analysis_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd



In [17]:
df=pd.read_csv("IMDB_Dataset.csv")

In [18]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [68]:

df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


In [19]:
df.shape

(5526, 2)

In [20]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

#### **Separate Dependent and Independent Feature**

In [22]:
X=df.drop('sentiment',axis=1)
y=pd.get_dummies(df['sentiment'],drop_first=True)
y

,positive
0,1
1,1
2,1
3,0
4,1
...,...
5521,0
5522,1
5523,0
5524,1


In [23]:
X.shape

(5526, 1)

##### Data cleaning( Lemmatize and removing stopwords)

In [24]:
# import dependensies
import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords

In [121]:
from tqdm import tqdm
lem=WordNetLemmatizer()
#nltk.download('stopwords')
#nltk.download('wordnet')

stemm=nltk.PorterStemmer()
lem=WordNetLemmatizer()
corpus=[]

for i in tqdm(range(0,len(df))):
  review=re.sub('[^a-zA-Z]',' ',df['review'][i])
  review.lower()
  review.split()
  review=[lem.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ''.join(review)
  corpus.append(review)

100%|██████████| 5526/5526 [14:11<00:00,  6.49it/s]


### Starting with ONE HOT and Padding apply on it

In [122]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [123]:
corpus[0]

'One f he her revewer h enne h fer wchng ju   Oz epe u ll be hke  The re rgh   h  excl wh hppene wh e  br    br   The fr hng h ruck e bu Oz w  brul n unflnchng cene f vlence  whch e n rgh fr he wr GO  Tru e  h  n  hw fr he fn here r   Th hw pull n punche wh regr  rug  ex r vlence  I  hrcre  n he clc ue f he wr  br    br   I  clle OZ  h  he nckne gven  he Owl Mxu Secur Se Penenr  I fcue nl n Eerl C  n experenl ecn f he prn where ll he cell hve gl frn n fce nwr   prvc  n hgh n he gen  E C  he  n  Arn  Mul  gng  Ln  Chrn  Iln  Irh n re     cuffle  eh re  g elng n h greeen re never fr w  br    br   I wul  he n ppel f he hw  ue  he fc h  ge where her hw wuln  re  Frge pre pcure pne fr nre uence  frge chr  frge rnce   OZ en  e run  The fr epe I ever w ruck e   n  w urrel  I culn   I w re fr   bu  I wche re  I evelpe  e fr Oz  n g ccue  he hgh level f grphc vlence  N ju vlence  bu njuce  crke gur wh ll be l u fr  nckel  ne wh ll kll n rer n ge w wh   well nnere  le cl ne beng urne n prn bche 

In [124]:

#Applied one hot on text
voc_size=500

onehot_repr=[one_hot(word,voc_size) for word in corpus]



In [125]:
#now we adding padding on it

sent_length=1000

embedded_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_doc.shape

(5526, 1000)

In [126]:
#making model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout


model=Sequential()
model.add(Embedding(voc_size,40,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

In [127]:
#compile our model

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1000, 40)          20000     
                                                                 
 dropout_2 (Dropout)         (None, 1000, 40)          0         
                                                                 
 lstm_2 (LSTM)               (None, 200)               192800    
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 213,001
Trainable params: 213,001
Non-trainable params: 0
_________________________________________________________________


In [128]:
import numpy as np

X=np.array(embedded_doc)
y=np.array(y)

In [129]:
X.shape

(5526, 1000)

In [130]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)

In [131]:
X_train.shape

(3702, 1000)

In [132]:
import tensorflow
ear_stop=tensorflow.keras.callbacks.EarlyStopping(patience=20,verbose=1)

In [134]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
58/58 [==============================] - 4s 76ms/step - loss: 0.6742 - accuracy: 0.5832 - val_loss: 0.6802 - val_accuracy: 0.5625
Epoch 2/50
58/58 [==============================] - 5s 86ms/step - loss: 0.6474 - accuracy: 0.6297 - val_loss: 0.6667 - val_accuracy: 0.5833
Epoch 3/50
58/58 [==============================] - 4s 76ms/step - loss: 0.6071 - accuracy: 0.6715 - val_loss: 0.6289 - val_accuracy: 0.6464
Epoch 4/50
58/58 [==============================] - 5s 86ms/step - loss: 0.5605 - accuracy: 0.7199 - val_loss: 0.6060 - val_accuracy: 0.6743
Epoch 5/50
58/58 [==============================] - 4s 74ms/step - loss: 0.5382 - accuracy: 0.7361 - val_loss: 0.6184 - val_accuracy: 0.6820
Epoch 6/50
58/58 [==============================] - 5s 84ms/step - loss: 0.5135 - accuracy: 0.7547 - val_loss: 0.6055 - val_accuracy: 0.6793
Epoch 7/50
58/58 [==============================] - 5s 84ms/step - loss: 0.4961 - accuracy: 0.7609 - val_loss: 0.5850 - val_accuracy: 0.7039
Epoch 8/50
58

In [135]:
from sklearn.metrics import classification_report,accuracy_score

y_pred=model.predict(X_test)
y_pred=np.where(y_pred>0.6,1,0)

print("accuracy we got is",accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))

57/57 [==============================] - 1s 19ms/step
accuracy we got is 0.6447368421052632
              precision    recall  f1-score   support

           0       0.63      0.73      0.68       929
           1       0.66      0.56      0.61       895

    accuracy                           0.64      1824
   macro avg       0.65      0.64      0.64      1824
weighted avg       0.65      0.64      0.64      1824

